In [2]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.6 MB/s eta 0:00:00a 0:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [1]:
from IPython.display import display, Image, Audio

import cv2
import base64
import time
from openai import OpenAI
import os
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


In [2]:
video = cv2.VideoCapture("/home/flowers-user/Downloads/discoveries/2024-06-06T15:13_exp_0_idx_276_seed_42/fecb27c5e2f6049b97554b79e1dfcd16f4c0bd3b.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")

5000 frames read.


In [20]:
len(base64Frames[0::500])

10

In [ ]:
NB_FRAMES = 10

In [ ]:
p= """Here's a series of images spaced one second apart.
With the eye of an expert biologist, mathematician, computer scientist, physicist, chemist, naturalist, historical and contemporary art, describe in the same time precisely, qualitatively and quantitatively, at high and low level, the following process and intricate patterns.
Don't just say 'complex' or 'detailed', be more precise by using a tailored vocabulary to describe spatial and temporal patterns (even from one frame to the other), and making analogies with real-world objects or phenomena.
Don't talk about colors since it's a grayscale image.
If an image is just a black or blank screen, just say 'black' or 'blank'.
Changes between frames are considered fast. 
Start directly with:
{Frame number}: {Contextual description}
"""

In [25]:
p2="""Here's a series of images spaced one second apart.
With the eye of an expert biologist, mathematician, computer scientist, physicist, chemist, naturalist, historical and contemporary art, associate a lexical field at the same time precise, qualitative and quantitative, describing at high and low level, the following process and intricate patterns.
Don't just say 'complex' or 'detailed', be more precise by using a tailored vocabulary to describe spatial and temporal patterns (even from one frame to the other), and making analogies with real-world objects or phenomena.
Don't talk about colors since it's a grayscale image.
If an image is just a black or blank screen, just say 'black' or 'blank'.
Changes between frames are considered fast. 
Start directly with:
Frame_{number}: {Detailed contextual lexical field}
"""

In [26]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            p2
           ,
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[500::500]
                 ),
        ],
    },
]
params = {
    "model": "gpt-4o",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 1000,
}

In [27]:
result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

Frame_1: The pattern is centrally localized, resembling a quadrilateral geometric shape with rounded edges, similar to a four-lobed star or clover, with blurred boundaries suggesting diffusion processes, possibly indicative of biological cell division or crystallographic symmetry initiation.

Frame_2: The previous structure has bifurcated into a more intricate configuration, showing a repetition of smaller clover-like shapes around a central void, reminiscent of fractal-like propagation or cellular automata growth dynamics, emphasizing geometric replication.

Frame_3: Increased complexity is observed as the smaller clover-like shapes multiply and arrange in a square lattice configuration, forming an array that suggests a higher-order symbiotic pattern or lattice formation commonly seen in crystal growth or molecular aggregation.

Frame_4: The structure further develops showing a central circular pattern with radial extensions forming a network of interconnected nodes, resembling a cell

In [60]:
# now we embed the text to a vector
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

embedding = get_embedding(result.choices[0].message.content)
